In [1]:
# need python 3.12+ for typing
!python -V

Python 3.12.6


In [2]:
%load_ext autoreload
%autoreload 2

In [54]:
from src.sokoban_solver import (
    SokobanEdge,
    SokobanNode,
    Solver,
    BFSQueue,
    PriorityQueue,
)
from src.heuristics import (
    f_priority_length,
    f_priority_l1_naive,
    f_priority_l1_symmetric_naive,
    f_priority_l1_matching,
    f_priority_dijkstra,
    f_priority_dijkstra_stuck_basic,
    f_priority_dlsb,
    get_f_priority_sbp,
)
from src.data_structures import Path
from boards.utilities import load_json_to_dict

In [35]:
A = np.array([[1, 2], [3, 4]])

In [37]:
idx, jdx = linear_sum_assignment(A)

In [38]:
A[idx, jdx]

array([1, 4])

## Demo

In [91]:
# loading a puzzle
json_dict = load_json_to_dict('boards/board_v=1&seed=1565730199&level=5.json')
node = SokobanNode(json_dict['airs'], json_dict['boxes'], json_dict['goals'], json_dict['player'])
print(node)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █         ● █ █ █
█ █ █ █ █ · █ █ █   █ █ █
█ █ █ █ █     █ █ □ █ █ █
█ █               □   █ █
█ █   █ █   █   █ █   █ █
█ █   █ █   █         █ █
█ █   █ █     ■ □ █ █ █ █
█ █           ·   █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █


In [92]:
# solving it: status code 200 means success!
f_priority_sbp = get_f_priority_sbp(node)
solver = Solver(PriorityQueue(f_priority_sbp), node)
solver.solve()

200

In [93]:
# here's the solution
print(solver.solution)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █         ● █ █ █
█ █ █ █ █ · █ █ █   █ █ █
█ █ █ █ █     █ █ □ █ █ █
█ █               □   █ █
█ █   █ █   █   █ █   █ █
█ █   █ █   █         █ █
█ █   █ █     ■ □ █ █ █ █
█ █           ·   █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █         · █ █ █
█ █ █ █ █ · █ █ █   █ █ █
█ █ █ █ █     █ █ □ █ █ █
█ █             □ ○   █ █
█ █   █ █   █   █ █   █ █
█ █   █ █   █         █ █
█ █   █ █     ■ □ █ █ █ █
█ █           ·   █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █         · █ █ █
█ █ █ █ █ · █ █ █   █ █ █
█ █ █ █ █     █ █ □ █ █ █
█ █           □ ○     █ █
█ █   █ █   █   █ █   █ █
█ █   █ █   █         █ █
█ █   █ █     ■ □ █ █ █ █
█ █           ·   █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █

In [94]:
path = Path(node, [], node)
f_priority_sbp(path)

np.float64(12.0)

## Scratchwork

In [81]:
from src.Tests.test_sokoban_solver import node_1_1565730199_5
node = node_1_1565730199_5

In [82]:
solver1 = Solver(PriorityQueue(f_priority_dlsb), node)
solver1.solve()

200

In [83]:
solver2 = Solver(PriorityQueue(get_f_priority_sbp(node)), node)
solver2.solve()

200

In [84]:
solver1.n_iters_overall

59

In [85]:
solver2.n_iters_overall

56

In [86]:
from src.Tests.test_sokoban_solver import node_1_1565730199_11
node = node_1_1565730199_11

In [87]:
# still can't solve this!
solver = Solver(PriorityQueue(get_f_priority_sbp(node)), node)
solver.solve(max_seconds=10)

301

In [90]:
# I think I need a additional stuckness heuristics
# definitely for weakly stuck boxes
# but ideally also something for tunnel areas...idk how
print(solver.queue[0])

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   □ · □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ · ○     █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   ○ ■ □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ ·       █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █     ■ □   █ █
█ █ · □ □   ■ ·     █ █
█ █   □ ○   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ ·       █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █     ■ □   █ █
█ █ · □ □   ■ ·     █ █
█ █ □ ○     █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ ·       █

## To do

**General**
- Better solution printing (Path objects)
    - Printing moves
    - Animating? .visualize() function?

**Generator**
- Figure out how the papers are doing things
- Implement something

**Solver**
- Implement more heuristics
    - Box pushing heuristic:
        - Upgrade of Dijkstra
        - Solving subpuzzles with just one box
        - Chained heuristics: solve subpuzzles with Dijkstra
        - **Also, this might be *really* slow: we have to consider each box/goal matchup to then do the linprog to find the smallest number of pushes. So that's n^2 subpuzzles to solve! It's probably best to just come up with pruning conditions based on how a box can be pushed (ex. if it's stuck in a dead-end hallway/component).**
        - Stuck box handling
            - Start with a stuck boxes pruning heuristic
            - Consider stuck-on-goal boxes to be equivalent to walls
            - Implement things reasonably flexibly: so if the notion of stuck (ex. basic vs "conditionally stuck but actually stuck") changes, it can still work
        - Include timeouts
            - How to handle these?
            - Fall back to Dijkstra?
            - Tuple priority with the first entry indicating timeout?
            - Option to repeat with longer timeouts on subsequent runs?
            - Tagging queue entries with solver objects so I can easily pick up heuristic evals once I exhaust more clearly viable paths?
            - **A general framework that manages priority calculation priorities (for multiple possible heuristics)? Maybe a secondary queue for that?**
    - Brainstorm more ideas
        - I should do more puzzles by hand to figure out next gen heuristics
- Ponder misc ideas
    - What would it look like to work neural nets into the problem? As the sole heuristic? For blending heuristics? For a CNN? What's the right learning framework? Reinforcement?
    - Splitting into sub-puzzles?